<h1>Design Patterns for ML projects</h1>

<h2>(1) Builder pattern (model.py)</h2>
    
<h7> Constructs a complex object (like a neural network architecture) step-by-step, allowing for flexible and customizable configurations.
     </h7>

In [ ]:

import torch
import torch.nn as nn

class ModelBuilder(nn.Module):
    def __init__(self):
        super(SimpleModelBuilder, self).__init__()
        self.layers = nn.ModuleList()
        self.input_shape = None  # Initialize input shape as None

    def add_layer(self, units, activation=None):
        if self.input_shape is None:
            raise ValueError("Input shape not set. Use set_input_shape before adding layers.")

        # Adding a fully connected layer
        self.layers.append(nn.Linear(self.input_shape, units))
        
        # If activation is provided, use the activation function
        if activation:
            self.layers.append(getattr(nn, activation)())

        # Update input shape for the next layer
        self.input_shape = units
        return self

    def set_input_shape(self, input_shape):
        self.input_shape = input_shape
        return self

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

# Usage
model = (ModelBuilder()
         .set_input_shape(28 * 28)  # Setting input shape for flattened 28x28 images
         .add_layer(128, 'ReLU')     # Adding hidden layer
         .add_layer(10, 'Softmax'))  # Adding output layer with softmax

<h2> (2) Reusable Training loop </h2>

In [ ]:
def train_loop(model, optimizer, loss_fn, dataloader, epochs):  
    for epoch in range(epochs):                     # epoch loop
        for inputs, targets in dataloader:          # batch loop
            optimizer.zero_grad()                     # clears the gradients from previous batch
            outputs = model(inputs)                   # forward pass
            loss = loss_fn(outputs, targets)          # compute loss
            loss.backward()                           # back propagation
            optimizer.step()                          # update params
        print(f"Epoch {epoch + 1}, Loss: {loss.item()}")


train_loop(model, optimizer, loss_fn, train_loader, epochs=10)

<h2> (3) Decorator pattern </h2>

<h7> Build a wrapper around a function in a modular and clean fashion.</h7>

In [ ]:
# This is an example of wrapper around a function. Apply a wrapper around a class is also possible.

import time

def log_training(func):
    def wrapper(*args, **kwargs):       # **kwargs (keyword arguments captures the key-value pairs)
        start_time = time.time()
        print(f"Starting training: {func.__name__}...")
        
        result = func(*args, **kwargs)  # Call actual training function
        
        end_time = time.time()
        print(f"Completed training: {func.__name__}. Time taken: {end_time - start_time:.2f}s")
        return result
    return wrapper

@log_training                          # this wraps the original function 'train_loop' with 'log_training'
def train_loop(model, optimizer, loss_fn, dataloader, epochs):
    for epoch in range(epochs):
        epoch_loss = 0
        for inputs, targets in dataloader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = loss_fn(outputs, targets)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        
        print(f"Epoch {epoch + 1}, Loss: {epoch_loss:.4f}")

# Assuming model, optimizer, loss_fn, and dataloader are predefined
train_loop(model, optimizer, loss_fn, dataloader, epochs=5)

<h2>(4) Facade pattern </h2>
    
<h7> Facade builds end-to-end pipelines for ML models, hiding the intricacy of each component (data loading, feature engineering, model training, evaluation) 
     </h7>

In [ ]:
class DataLoader:
    def load_data(self):
        data = "raw_data"  # Placeholder for actual data
        return data

class Preprocessor:
    def preprocess_data(self, data):
        processed_data = "processed_data"  # Placeholder for actual processed data
        return processed_data

class ModelTrainer:
    def train_loop(self, model, optimizer, loss_fn, dataloader, epochs):
        for epoch in range(epochs):                     # epoch loop
            for inputs, targets in dataloader:          # batch loop
                optimizer.zero_grad()                   # clears gradients from previous batch
                outputs = model(inputs)                 # forward pass
                loss = loss_fn(outputs, targets)        # compute loss
                loss.backward()                         # backpropagation
                optimizer.step()                        # update parameters
            print(f"Epoch {epoch + 1}, Loss: {loss.item()}")
        print("Training complete")
        return model

class ModelPipelineFacade:
    def __init__(self, model, optimizer, loss_fn, dataloader, epochs=10):
        self.data_loader = DataLoader()
        self.preprocessor = Preprocessor()
        self.trainer = ModelTrainer()
        self.model = model
        self.optimizer = optimizer
        self.loss_fn = loss_fn
        self.dataloader = dataloader
        self.epochs = epochs

    def run_pipeline(self):
        # Step 1: Load data
        data = self.data_loader.load_data()
        
        # Step 2: Preprocess data
        processed_data = self.preprocessor.preprocess_data(data)
        
        # Step 3: Train model
        trained_model = self.trainer.train_loop(
            self.model, self.optimizer, self.loss_fn, self.dataloader, self.epochs
        )
        
        print("Pipeline complete")
        return trained_model

# Usage
pipeline = ModelPipelineFacade(model, optimizer, loss_fn, train_loader, epochs=10)
pipeline.run_pipeline()

<h3>(5) Observer pattern </h3>
    